Adhesion Demo
==============

This example demonstrates how to specify cell adhesion on the basis of molecular species.

Adjust the sliders to change the binding energy of two molecules, each of which is only on the surface of a particular
cell type. A third cell type has neither molecule.

Basic setup
------------
An interactive CC3D simulation can be initialized from a list of core specs.
Start a list of core specs that define the simulation by specifying a two-dimensional simulation
with a 100x100 lattice and second-order Potts neighborhood, and metadata to use multithreading

In [1]:
from cc3d.core.PyCoreSpecs import Metadata, PottsCore

dim_x = dim_y = 100
specs = [
    Metadata(num_processors=4),
    PottsCore(dim_x=dim_x,
              dim_y=dim_y,
              neighbor_order=2,
              boundary_x="Periodic",
              boundary_y="Periodic")
]

Cell Types
-----------

Define three cell types called "T1" through "T3".

In [2]:
from cc3d.core.PyCoreSpecs import CellTypePlugin

cell_types = ["T1", "T2", "T3"]
specs.append(CellTypePlugin(*cell_types))

Volume Constraint
------------------

Assign a volume constraint to all cell types.

In [3]:
from cc3d.core.PyCoreSpecs import VolumePlugin

volume_specs = VolumePlugin()
for ct in cell_types:
    volume_specs.param_new(ct, target_volume=25, lambda_volume=2)
specs.append(volume_specs)

Adhesion
---------

Assign uniform adhesion to all cells, and additional adhesion by molecular species.

In [4]:
from cc3d.core.PyCoreSpecs import ContactPlugin, AdhesionFlexPlugin

contact_specs = ContactPlugin(neighbor_order=2)
for idx1 in range(len(cell_types)):
    contact_specs.param_new(type_1="Medium", type_2=cell_types[idx1], energy=16)
    for idx2 in range(idx1, len(cell_types)):
        contact_specs.param_new(type_1=cell_types[idx1], type_2=cell_types[idx2], energy=16)
specs.append(contact_specs)

adhesion_specs = AdhesionFlexPlugin(neighbor_order=2)
adhesion_specs.density_new(molecule="M1", cell_type="T1", density=1.0)
adhesion_specs.density_new(molecule="M2", cell_type="T2", density=1.0)
formula = adhesion_specs.formula_new(formula_name='Binary')
formula.param_set("M1", "M1", -10.0)
formula.param_set("M1", "M2", 0.0)
formula.param_set("M2", "M2", 10.0)
specs.append(adhesion_specs)

Cell Distribution Initialization
---------------------------------

Initialize cells over the entire domain.

In [5]:
from cc3d.core.PyCoreSpecs import UniformInitializer

unif_init_specs = UniformInitializer()
unif_init_specs.region_new(width=5, pt_min=(0, 0, 0), pt_max=(dim_x, dim_y, 1),
                           cell_types=["T1", "T1", "T2", "T2", "T3"])
specs.append(unif_init_specs)

In [17]:
# import json
# with open('AdhesionDemoSpec.json', 'w') as f:
#     json.dump(specs, f, indent=4)

for spec in specs:
    print(spec.__dict__)

{'spec_dict': {'num_processors': 4, 'debug_output_frequency': 0}, '_el': <cc3d.core.XMLUtils.ElementCC3D object at 0x000001AA138687C0>}
{'spec_dict': {'dim_x': 100, 'dim_y': 100, 'dim_z': 1, 'steps': 0, 'anneal': 0, 'fluctuation_amplitude': 10.0, 'fluctuation_amplitude_function': 'Min', 'boundary_x': 'Periodic', 'boundary_y': 'Periodic', 'boundary_z': 'NoFlux', 'neighbor_order': 2, 'debug_output_frequency': 0, 'random_seed': None, 'lattice_type': 'Cartesian', 'offset': 0, 'energy_function_calculator': None}, '_el': <cc3d.core.XMLUtils.ElementCC3D object at 0x000001AA13868B50>}
{'spec_dict': {'cell_types': [('Medium', 0, False), ('T1', 1, False), ('T2', 2, False), ('T3', 3, False)]}, '_el': <cc3d.core.XMLUtils.ElementCC3D object at 0x000001AA138691E0>}
{'spec_dict': {'params': {'T1': <cc3d.core.PyCoreSpecs.VolumeEnergyParameter object at 0x000001AA0F8B0FA0>, 'T2': <cc3d.core.PyCoreSpecs.VolumeEnergyParameter object at 0x000001AA0F8B1F60>, 'T3': <cc3d.core.PyCoreSpecs.VolumeEnergyParamet

Simulation Launch
------------------

Initialize a CC3D simulation service instance and register all simulation specification.

In [6]:
from cc3d.CompuCellSetup.CC3DCaller import CC3DSimService

cc3d_sim = CC3DSimService()
cc3d_sim.register_specs(specs)
cc3d_sim.run()
cc3d_sim.init()
cc3d_sim.start()

a:\conda_envs\cc3d_env\lib\site-packages\cc3d\core\Configuration\SettingUtils.py:248: UserWarning: Setting not set: PlayerSizes
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')
a:\conda_envs\cc3d_env\lib\site-packages\cc3d\core\Configuration\SettingUtils.py:248: UserWarning: Setting not set: PlayerSizesFloating
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')
a:\conda_envs\cc3d_env\lib\site-packages\cc3d\core\Configuration\SettingUtils.py:248: UserWarning: Setting not set: RecentSimulations
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')


WILL RUN SIMULATION FROM BEGINNING


True

Steering
---------

Add sliders to adjust molecular adhesion model parameters during simulation execution.

In [ ]:
import ipywidgets

def _cb_m1_m1(change):
    if change['name'] == 'value':
        adhesion_specs.formula['Binary']['M1']['M1'] = change.new
        adhesion_specs.steer()

def _cb_m1_m2(change):
    if change['name'] == 'value':
        adhesion_specs.formula['Binary']['M1']['M1'] = change.new
        adhesion_specs.steer()

def _cb_m2_m2(change):
    if change['name'] == 'value':
        adhesion_specs.formula['Binary']['M2']['M2'] = change.new
        adhesion_specs.steer()

slider_m1_m1 = ipywidgets.FloatSlider(
    value=adhesion_specs.formula['Binary']['M1']['M1'],
    min=-10,
    max=10,
    step=0.1,
    continuous_update=False,
    description='M1-M1'
)
slider_m1_m2 = ipywidgets.FloatSlider(
    value=adhesion_specs.formula['Binary']['M1']['M2'],
    min=-10,
    max=10,
    step=0.1,
    continuous_update=False,
    description='M1-M2'
)
slider_m2_m2 = ipywidgets.FloatSlider(
    value=adhesion_specs.formula['Binary']['M2']['M2'],
    min=-10,
    max=10,
    step=0.1,
    continuous_update=False,
    description='M2-M2'
)

slider_m1_m1.observe(_cb_m1_m1, names='value')
slider_m1_m2.observe(_cb_m1_m2, names='value')
slider_m2_m2.observe(_cb_m2_m2, names='value')

Visualization
--------------

Show a single frame to visualize simulation data as it is generated and all steering widgets.

In [8]:
from IPython.display import display

cc3d_sim.visualize(plot_freq=10).show()
display(slider_m1_m1)
display(slider_m1_m2)
display(slider_m2_m2)
display(cc3d_sim.jupyter_run_button())

CC3DViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=600)

FloatSlider(value=-10.0, continuous_update=False, description='M1-M1', max=10.0, min=-10.0)

FloatSlider(value=0.0, continuous_update=False, description='M1-M2', max=10.0, min=-10.0)

FloatSlider(value=10.0, continuous_update=False, description='M2-M2', max=10.0, min=-10.0)

ToggleButton(value=False, description='Run Simulation')